# Models

In [ ]:
real = Variable(torch.randn([2,1,32,32,32]).cuda())
# real loss
netD.zero_grad()
errD_real = step_D(real, mone)
# fake loss
noise = Variable(torch.randn(2, 128).cuda(), volatile=True)
fake = Variable(netG(noise).data)
errD_fake = step_D(fake, one)
# Gradient penalty (improved wgan paper):
gradient_penalty = calc_gradient_penalty(netD, real.data, fake.data)
gradient_penalty.backward()

In [14]:
netG = G(128).cuda()
noise = Variable(torch.randn(2, 128).cuda(), volatile=True)
fake = Variable(netG(noise).data)
fake.size()

torch.Size([2, 1, 32, 32, 32])

In [15]:
netD = D(128).cuda()
# netD(fake).size()
errD_fake = step_D(fake, one)

In [20]:
real = Variable(torch.randn([2,1,32,32,32]).cuda())
gradient_penalty = calc_gradient_penalty(netD, real.data, fake.data)

torch.Size([2, 1, 32, 32, 32]) torch.Size([2, 1])


In [21]:
gradient_penalty.backward()

RuntimeError: ConvNdBackward: expected Variable at argument 0 (got None)

In [19]:
from torch.autograd import grad

def calc_gradient_penalty(netD, real_data, fake_data):
    alpha = torch.rand(real_data.size()[0],1,1,1,1)
    alpha = alpha.expand(real_data.size()).cuda()
    interpolates = (alpha * real_data) + ((1 - alpha) * fake_data)
    interpolates = Variable(interpolates, requires_grad=True)
    disc_interpolates = netD(interpolates)
    print(interpolates.size(), disc_interpolates.size())
    inputs = [interpolates] + list(netD.parameters())
    
    gradients = grad(outputs=disc_interpolates, inputs=inputs,
                     grad_outputs=torch.ones(disc_interpolates.size()).cuda(),
                     create_graph=True, retain_graph=True, only_inputs=True)[0]

    gradient_penalty = ((gradients[0].norm(2, dim=1) - 1) ** 2).mean() * LAMBDA
    
    for gradi in gradients[1:]:
        gradient_penalty += 0 * gradi.view(-1)[0]

    return gradient_penalty

In [ ]:
def calc_gradient_penalty(netD, real_data, fake_data):
    alpha = torch.rand(real_data.size()[0],1,1,1,1)
    alpha = alpha.expand(real_data.size()).cuda()
    interpolates = (alpha * real_data) + ((1 - alpha) * fake_data)
    interpolates = Variable(interpolates, requires_grad=True)
    disc_interpolates = netD(interpolates)
    print(interpolates.size(), disc_interpolates.size())
    gradients = grad(outputs=disc_interpolates, inputs=interpolates,
                     grad_outputs=torch.ones(disc_interpolates.size()).cuda(),
                     create_graph=True, retain_graph=True, only_inputs=True)[0]
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * LAMBDA

    return gradient_penalty

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
from torch.autograd import Variable

class D(nn.Module):
    def __init__(self, features):
        super(D, self).__init__()
        self.l1 = nn.Conv3d(1, 2, 4, 2, 1, bias=False)
        self.l2 = nn.Conv3d(2, 4, 4, 2, 1, bias=False)
        self.l3 = nn.Conv3d(4, 8, 4, 2, 1, bias=False)
        self.l4 = nn.Conv3d(8, 8, 4, 2, 1, bias=False)
        self.l5 = nn.Linear(8*2*2*2, 128)
        self.l6 = nn.Linear(128, 1)
        
    def forward(self, x):
        out = self.l1(x)
        out = self.l2(out)
        out = self.l3(out)
        out = self.l4(out)
        out = out.view([-1,8*2*2*2])
        out = self.l5(out)
        out = self.l6(out)
        
        return out

class G(nn.Module):
    def __init__(self, features):
        super(G, self).__init__()
        self.features = features
        self.l1 = nn.Linear(features, 4*4*4)
        self.l2 = nn.ConvTranspose3d(1, 4, 4, 2, 1, bias=False)
        self.l3 = nn.ConvTranspose3d(4, 8, 4, 2, 1, bias=False)
        self.l4 = nn.ConvTranspose3d(8, 1, 4, 2, 1, bias=False)

    def forward(self, x):
        out = self.l1(x)
        out = out.view([-1,1,4,4,4])
        out = self.l2(out)
        out = self.l3(out)
        out = self.l4(out)
        
        return out

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
from torch.autograd import Variable

class D(nn.Module):
    def __init__(self, features):
        super(D, self).__init__()
        self.l1 = nn.Conv2d(1, 2, 4, 2, 1, bias=False)
        self.l2 = nn.Conv2d(2, 4, 4, 2, 1, bias=False)
        self.l3 = nn.Conv2d(4, 8, 4, 2, 1, bias=False)
        self.l4 = nn.Conv2d(8, 8, 4, 2, 1, bias=False)
        self.l5 = nn.Linear(8*2*2, 128)
        self.l6 = nn.Linear(128, 1)
        
    def forward(self, x):
        out = self.l1(x)
        out = self.l2(out)
        out = self.l3(out)
        out = self.l4(out)
        out = out.view([-1,8*2*2])
        out = self.l5(out)
        out = self.l6(out)
        
        return out

class G(nn.Module):
    def __init__(self, features):
        super(G, self).__init__()
        self.features = features
        self.l1 = nn.Linear(features, 4*4)
        self.l2 = nn.ConvTranspose2d(1, 4, 4, 2, 1, bias=False)
        self.l3 = nn.ConvTranspose2d(4, 8, 4, 2, 1, bias=False)
        self.l4 = nn.ConvTranspose2d(8, 1, 4, 2, 1, bias=False)

    def forward(self, x):
        out = self.l1(x)
        out = out.view([-1,1,4,4])
        out = self.l2(out)
        out = self.l3(out)
        out = self.l4(out)
        
        return out

# Improved Wasserstein GAN in Pytorch

In [4]:
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms, utils
%matplotlib inline

First, we, set up batch size, image size, and size of noise vector:

In [5]:
bs,sz,nz = 256,64,128
LAMBDA = 10
PATH_G = 'iwgancv5_g'
PATH_D = 'iwgancv5_d'

In [ ]:
PATH = '/home/m20163692/data/lsun'
data = datasets.LSUN(db_path=PATH, classes=['church_outdoor_train'],
    transform=transforms.Compose([
        transforms.Scale(sz),
        transforms.CenterCrop(sz),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]))

Even parallel processing is handling automatically by torch-vision.

In [ ]:
dataloader = torch.utils.data.DataLoader(data, bs, True, num_workers=8, pin_memory=True)
n = len(dataloader); n

In [ ]:
next(iter(dataloader))[0].size()

Our activation function will be `tanh`, so we need to do some processing to view the generated images.

In [ ]:
def show(img, fs=(8,8)):
    plt.figure(figsize = fs)
    plt.imshow(np.transpose((img/2+0.5).clamp(0,1).numpy(), (1,2,0)), interpolation='nearest')
    plt.show()

## Create model

In [ ]:
import torch.nn as nn
from torch import optim
from torch.autograd import Variable, grad

Pytorch uses `module.apply()` for picking an initializer.

In [ ]:
netG = G(nz).cuda()
netG.apply(weights_init);

netD = D(nz).cuda()
netD.apply(weights_init);

Just some shortcuts to create tensors and variables.

In [6]:
from torch import FloatTensor as FT

def Var(*params):
    return Variable(FT(*params).cuda())

def create_noise(b, volatile=False): 
    return Variable(torch.randn(b, nz).cuda(), volatile=volatile)
# Variable(FT(b, nz).cuda().normal_(0, 1), volatile=volatile)

In [7]:
# Input placeholder
input = Var(bs, 3, sz, nz)
# Fixed noise used just for visualizing images when done
fixed_noise = create_noise(bs)
# The numbers 1 and -1
one = torch.FloatTensor([1]).cuda()
mone = one * -1

Adam parameters from paper

In [ ]:
# TODO ADAM
optimizerD = optim.Adam(netD.parameters(), lr = 1e-4, betas=(0.5, 0.9))
optimizerG = optim.Adam(netG.parameters(), lr = 1e-4, betas=(0.5, 0.9))

One forward step and one backward step for D

In [ ]:
data_iter = iter(dataloader)
real = Variable(next(data_iter)[0].cuda())
real.size()

In [ ]:
fake = netG(create_noise(real.size()[0]))
fake.size()

In [ ]:
# alpha = torch.rand(bs,1,1,1).cuda()
# alpha = alpha.expand_as(real)
# alpha.size()

In [ ]:
# interpolates = (alpha * real.data) + ((1 - alpha) * fake.data)
# interpolates.size()

In [ ]:
# interpolates = torch.autograd.Variable(interpolates, requires_grad=True)
# D_interpolates = netD(interpolates)
# D_interpolates.size()

In [ ]:
# gradients = grad(D_interpolates, interpolates, create_graph=True)[0]
# gradients.size()

In [ ]:
# gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * LAMBDA
# gradient_penalty.size()

In [ ]:
# D_loss = D_fake - D_real + LAMBDA * gradient_penalty
# D_loss.backward()
# optimizerD.step()

In [8]:
def step_D(v, init_grad):
    err = netD(v).mean()
    err.backward(init_grad)

    return err

def calc_gradient_penalty(netD, real_data, fake_data):
    alpha = torch.rand(real_data.size()[0],1,1,1)
    alpha = alpha.expand(real_data.size()).cuda()
    interpolates = (alpha * real_data) + ((1 - alpha) * fake_data)
    interpolates = Variable(interpolates, requires_grad=True)
    disc_interpolates = netD(interpolates)
    print(interpolates.size(), disc_interpolates.size())
    gradients = grad(outputs=disc_interpolates, inputs=interpolates,
                     grad_outputs=torch.ones(disc_interpolates.size()).cuda(),
                     create_graph=True, retain_graph=True, only_inputs=True)[0]
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * LAMBDA

    return gradient_penalty

def make_trainable(net, val): 
    for p in net.parameters(): p.requires_grad = val

def train(niter, first=True):
    netG.train()
    gen_iterations = 0
    save_flag = 1
    for epoch in range(niter):
        t0 = time()
        data_iter = iter(dataloader)
        i = 0
        
        while i < n:
            make_trainable(netD, True)
            #d_iters = (50 if first and (gen_iterations < 5) or gen_iterations % 500 == 0 
            #           else 5)
            d_iters = 5
            j = 0
            while j < d_iters and i < n:
                j += 1; i += 1
                print(f'{i:04}/{n:04} - G_iterations: {gen_iterations:04}', end='\r')
                real = Variable(next(data_iter)[0].cuda())
                # real loss
                netD.zero_grad()
                errD_real = step_D(real, mone)
                # fake loss
                noise = create_noise(real.size()[0], volatile=True)
                fake = Variable(netG(noise).data)
                errD_fake = step_D(fake, one)
                # Gradient penalty (improved wgan paper):
                gradient_penalty = calc_gradient_penalty(netD, real.data, fake.data)
                gradient_penalty.backward()
                errD = errD_fake - errD_real + gradient_penalty
                optimizerD.step()
#                 print('D: %.3f | DF: %.3f | DR: %.3f | GP: %.3f' %
#                       (errD.data[0], errD_fake.data[0], errD_real.data[0], gradient_penalty.data[0]))
                
            make_trainable(netD, False)
            netG.zero_grad()
            errG = -step_D(netG(create_noise(bs)), mone)
            optimizerG.step()
            gen_iterations += 1
            if save_flag == -1:
                p_g = PATH_G + '_temp.pkl'
                p_d = PATH_D + '_temp.pkl'
            else:
                p_g = PATH_G + '.pkl'
                p_d = PATH_D + '.pkl'
            
            torch.save(netG.state_dict(), p_g)
            torch.save(netD.state_dict(), p_d)
            save_flag *= -1
            
            with open('logs/log.txt', 'a') as f:
                f.writelines('[%d/%d][%d/%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f (%.2fs)\n' %
                             (epoch, niter, gen_iterations, n, errD.data[0], errG.data[0],
                              errD_real.data[0], errD_fake.data[0], time() - t0))
            
        print('[%d/%d][%d/%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f (%.2fs)' % (
            epoch, niter, gen_iterations, n,
            errD.data[0], errG.data[0], errD_real.data[0], errD_fake.data[0], time() - t0))

In [ ]:
%%time
train(50000, True) # First run
# train(10000, False)

In [ ]:
p_g = PATH_G + '.pkl'
p_d = PATH_D + '.pkl'
torch.save(netG.state_dict(), p_g)
torch.save(netD.state_dict(), p_d)

## View

In [ ]:
# LOAD
netG.load_state_dict(torch.load(PATH_G + '.pkl'))
netD.load_state_dict(torch.load(PATH_D + '.pkl'))

In [ ]:
# model mode eval
netG.eval()
fixed_noise = create_noise(64)
fake = netG(fixed_noise).data.cpu()
# show(random.choice(fake))
show(utils.make_grid(fake), (16,16))

## Check interpolation

In [ ]:
from IPython.display import HTML
from matplotlib import animation, rc

fig, ax = plt.subplots()

def animate(i):
    ax.imshow(np.transpose((interpolation[i]/2+0.5).clamp(0,1).numpy(), (1,2,0)), interpolation='nearest')

In [ ]:
idxs = [4,52]
noise = fixed_noise[idxs, :]
img = netG(noise).data.cpu()
show(utils.make_grid(img), (7,7))

In [ ]:
n_imgs = 100
diff = (noise[1].data - noise[0].data) / n_imgs
noise_interpolation = torch.arange(0, n_imgs+1).cuda()
noise_interpolation = noise_interpolation.repeat(nz, 1).transpose(0, 1)
noise_interpolation = noise[0].data + (diff * noise_interpolation)

interpolation = netG(Variable(noise_interpolation, volatile=True)).data.cpu()

In [ ]:
# show(utils.make_grid(interpolation, nrow=int(np.sqrt(n_imgs))), (16,16))

In [ ]:
anim = animation.FuncAnimation(fig, animate, frames=n_imgs, interval=100)
HTML(anim.to_html5_video())

In [ ]:
show(utils.make_grid(iter(dataloader).next()[0][:64]), (16,16))